In [1]:
import copy
import numpy as np
import pandas as pd
from fixedincomelib.data import DataCollection, Data1D, Data2D,build_yc_data_collection
from fixedincomelib.date import Date
from fixedincomelib.yield_curve import YieldCurve
from fixedincomelib.valuation import IndexManager
from fixedincomelib.valuation import ValuationEngineRegistry
from fixedincomelib.product import (ProductOvernightCapFloorlet, ProductOvernightSwaption, ProductOvernightCapFloor)
import QuantLib as ql
from fixedincomelib.builders import create_products_from_data1d
from fixedincomelib.builders import build_yc_calibration_basket
from fixedincomelib.product.product_display_visitor import RfrFutureVisitor, OvernightSwapVisitor
from fixedincomelib.utilities.risk_reporting import createValueReport, risk_vectors_to_df
from fixedincomelib.sabr import valuation_engine_sabr, SabrModel
from fixedincomelib.analytics import SABRCalculator


In [2]:
IndexManager.instance()

In [3]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.70],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [4]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [5]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]

In [6]:
yc = YieldCurve("2025-09-24", dc, build_methods)
print("Curve components:", yc.components.keys())

Curve components: dict_keys(['SOFR-1B'])


In [7]:
import pandas as pd

# Expiry x Tenor grid (years)
expiries = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0, 10.0]      # 1M, 3M, 6M, 1Y, 2Y, 5Y, 10Y
tenors   = [1.0, 2.0, 5.0, 10.0, 30.0]                 # 1Y, 2Y, 5Y, 10Y, 30Y

# ATM normal vols (rate units; 0.0070 = 70bp)
atm_normal = {
    1/12:{ 1:0.0090,  2:0.0086,  5:0.0079, 10:0.0072, 30:0.0066},
    0.25:{ 1:0.0086,  2:0.0082,  5:0.0074, 10:0.0068, 30:0.0062},
    0.5: { 1:0.0081,  2:0.0077,  5:0.0070, 10:0.0064, 30:0.0059},
    1.0: { 1:0.0074,  2:0.0070,  5:0.0063, 10:0.0059, 30:0.0055},
    2.0: { 1:0.0067,  2:0.0063,  5:0.0058, 10:0.0054, 30:0.0051},
    5.0: { 1:0.0060,  2:0.0057,  5:0.0053, 10:0.0050, 30:0.0048},
    10.0:{ 1:0.0056,  2:0.0053,  5:0.0050, 10:0.0047, 30:0.0045},
}

# SABR-style params that "look like a desk surface":

beta = 0.6    # normal SABR

def nu_surface(expiry, tenor):
    base = {1/12:1.20, 0.25:1.00, 0.5:0.85, 1.0:0.70, 2.0:0.55, 5.0:0.40, 10.0:0.30}[expiry]
    tenor_factor = {1.0:1.00, 2.0:0.95, 5.0:0.90, 10.0:0.85, 30.0:0.80}[tenor]
    return base * tenor_factor

def rho_surface(expiry, tenor):
    base = {1/12:-0.45, 0.25:-0.42, 0.5:-0.38, 1.0:-0.34, 2.0:-0.30, 5.0:-0.26, 10.0:-0.22}[expiry]
    tenor_adj = {1.0:0.00, 2.0:0.02, 5.0:0.04, 10.0:0.05, 30.0:0.06}[tenor]
    return base + tenor_adj

rows = []
for e in expiries:
    for t in tenors:
        rows.append([
            "SOFR-1B",
            float(e),
            float(t),
            float(atm_normal[e][t]),
            beta,
            float(nu_surface(e, t)),
            float(rho_surface(e, t)),
        ])

sabr_sofr = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])
sabr_sofr


,INDEX,AXIS1,AXIS2,NORMALVOL,BETA,NU,RHO
0,SOFR-1B,0.083333,1.000000,0.009000,0.600000,1.200000,-0.450000
1,SOFR-1B,0.083333,2.000000,0.008600,0.600000,1.140000,-0.430000
2,SOFR-1B,0.083333,5.000000,0.007900,0.600000,1.080000,-0.410000
3,SOFR-1B,0.083333,10.000000,0.007200,0.600000,1.020000,-0.400000
4,SOFR-1B,0.083333,30.000000,0.006600,0.600000,0.960000,-0.390000
5,SOFR-1B,0.250000,1.000000,0.008600,0.600000,1.000000,-0.420000
6,SOFR-1B,0.250000,2.000000,0.008200,0.600000,0.950000,-0.400000
7,SOFR-1B,0.250000,5.000000,0.007400,0.600000,0.900000,-0.380000
8,SOFR-1B,0.250000,10.000000,0.006800,0.600000,0.850000,-0.370000
9,SOFR-1B,0.250000,30.000000,0.006200,0.600000,0.800000,-0.360000


In [8]:
PARAMS = ["NORMALVOL","BETA","NU","RHO"]

sabr_objs = []
for idx_name, sub in sabr_sofr.groupby("INDEX"):
    for param in PARAMS:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0)
               .sort_index(axis=1)
        )
        sabr_objs.append(
            Data2D.createDataObject(
                data_type=param,          # stored lower() internally
                data_convention=idx_name, # e.g. "SOFR-1B"
                df=pivot
            )
        )

sabr_dc = DataCollection(sabr_objs)

In [9]:
value_date = "2025-09-24"

sabr_build_methods = [
    {
        "TARGET": idx_name,         # "SOFR-1B"
        "VALUES": param,            # "NORMALVOL", "BETA", "NU", "RHO"
        "INTERPOLATION": "LINEAR",
        "SHIFT": 0.0,
        "VOL_DECAY_SPEED": 0.2
    }
    for idx_name in sabr_sofr["INDEX"].unique()
    for param in PARAMS
]

sabr_model = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=sabr_dc,
    buildMethodCollection=sabr_build_methods,
    ycModel=yc
)

print(sabr_model.components.keys())

dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [10]:
np.set_printoptions(threshold=np.inf, linewidth=200, suppress=True)
sabr_model.jacobian()

array([[-100.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.  

In [11]:
swaption = ProductOvernightSwaption(
    optionExpiry="2026-09-24",
    swapStart="2026-09-24",
    swapEnd="2031-09-24",
    frequency="1Y",
    overnightIndex="SOFR-1B",
    optionType="PAYER",
    strikeRate=0.04,
    notional=10000000,
    longOrShort="LONG",
)

vp = {"SABR_METHOD": "hagan", "FUNDING INDEX": "SOFR-1B"}  # method will be forced to Hagan anyway
ve = ValuationEngineRegistry().new_valuation_engine(sabr_model, vp, swaption)

ve.calculateValue()
print("Swaption PV:", ve.value)

ve.calculateFirstOrderRisk()
g = np.asarray(ve.firstOrderRisk, dtype=float)
print(g)

a= ve.value[1]


Swaption PV: ['USD', np.float64(54235.42365401054)]
[  -13709.5098681    -13558.85591351   -13860.16382269   -13860.16382269   721813.41701887   713881.40144723   729745.43259051   729745.43259051   698625.89541921   698625.8954192    706303.10306118
   706303.10306118  2712250.22977443  2625273.4176067   2541327.18718485        0.                0.                0.                0.                0.                0.                0.
        0.                0.                0.                0.                0.                0.                0.                0.                0.                0.                0.
        0.                0.                0.                0.                0.                0.                0.                0.                0.                0.         15038878.5441149
   220411.97415275        0.                0.                0.           211815.19076218     3104.39400215        0.                0.                0.               

In [12]:
report = createValueReport(vp, sabr_model, swaption, request="all", space="pv")
g = report["param_risk"]
print(g.shape, g)

(166,) [    7765.92179309     7764.06565244     7876.05322472     7958.57676018      138.28931175      138.12488424       -1.33173303       -1.3445295       105.21565125      106.17843292      -36.33214936
      -36.65712025    14217.53854823    14748.12832859 -3046963.04048906       -0.               -0.               -0.               -0.               -0.               -0.               -0.
       -0.               -0.               -0.               -0.                0.                0.                0.                0.                0.                0.                0.
        0.                0.                0.                0.                0.                0.                0.                0.                0.                0.         15038878.5441149
   220411.97415275        0.                0.                0.           211815.19076218     3104.39400215        0.                0.                0.                0.                0.
        0.             

In [13]:
report_q = createValueReport(vp, sabr_model, swaption, request="all", space="quote")
dq = report_q["quote_risk"]
print(dq.shape)
print(dq)

(166,)
[    -7682.41817553     -7600.77386756     -7630.45592517     -7632.38619889      -131.3201435       -129.91680111         1.24067319         1.24092447       -96.22754309       -96.21261883
        32.63018697        32.62795467     52915.61160821     67458.04641932 -16433306.33685206        -0.                -0.                -0.                -0.                -0.
        -0.                -0.                -0.                -0.                -0.                -0.                 0.                 0.                 0.                 0.
         0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.                 0.
         0.                 0.                 0.          15038878.5441149     220411.97415275         0.                 0.                 0.            211815.19076218      3104.39400215
         0.                 0.                 0.             

In [14]:
pd.set_option("display.float_format", lambda x: f"{x:,.6f}")

In [15]:
df_report = risk_vectors_to_df(sabr_model,report_q, yc_index="SOFR-1B") 

df_report.head(5)

,block,index,param,expiry,tenor,node,pos,dPV_dModelParam,hedgeWeightPV,dPV_dQuote,abs_model,abs_weight,abs_quote
0,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M September 24th, 2025...",0,"-13,709.509868","7,765.921793","-7,682.418176","13,709.509868","7,765.921793","7,682.418176"
1,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M December 24th, 2025 ...",1,"-13,558.855914","7,764.065652","-7,600.773868","13,558.855914","7,764.065652","7,600.773868"
2,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M March 24th, 2026 x J...",2,"-13,860.163823","7,876.053225","-7,630.455925","13,860.163823","7,876.053225","7,630.455925"
3,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M June 24th, 2026 x Se...",3,"-13,860.163823","7,958.576760","-7,632.386199","13,860.163823","7,958.576760","7,632.386199"
4,YC,SOFR-1B,IFR,NaN,NaN,"RFR FUTURE SOFR-FUTURE-3M September 24th, 2026...",4,"721,813.417019",138.289312,-131.320143,"721,813.417019",138.289312,131.320143


In [16]:
MARKET_DF2 = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.71],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [17]:
data_objs2, dc2 = build_yc_data_collection(MARKET_DF2)

In [18]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
}]



In [19]:
yc2 = YieldCurve("2025-09-24", dc2, build_methods)
print("Curve components:", yc2.components.keys())

Curve components: dict_keys(['SOFR-1B'])


In [20]:
value_date = "2025-09-24"

sabr_build_methods = [
    {
        "TARGET": idx_name,         # "SOFR-1B"
        "VALUES": param,            # "NORMALVOL", "BETA", "NU", "RHO"
        "INTERPOLATION": "LINEAR",
        "SHIFT": 0.0,
        "VOL_DECAY_SPEED": 0.2
    }
    for idx_name in sabr_sofr["INDEX"].unique()
    for param in PARAMS
]

sabr_model2 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=sabr_dc,
    buildMethodCollection=sabr_build_methods,
    ycModel=yc2
)

print(sabr_model.components.keys())

dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [21]:
swaption = ProductOvernightSwaption(
    optionExpiry="2026-09-24",
    swapStart="2026-09-24",
    swapEnd="2031-09-24",
    frequency="1Y",
    overnightIndex="SOFR-1B",
    optionType="PAYER",
    strikeRate=0.04,
    notional=10000000,
    longOrShort="LONG",
)

vp = {"SABR_METHOD": "hagan", "FUNDING INDEX": "SOFR-1B"}  # method will be forced to Hagan anyway
ve = ValuationEngineRegistry().new_valuation_engine(sabr_model2, vp, swaption)

ve.calculateValue()

b = ve.value[1]

In [22]:
(b-a)/0.01

np.float64(7687.261468115321)

In [23]:
import pandas as pd

# Expiry x Tenor grid (years)
expiries = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0, 10.0]      # 1M, 3M, 6M, 1Y, 2Y, 5Y, 10Y
tenors   = [1.0, 2.0, 5.0, 10.0, 30.0]                 # 1Y, 2Y, 5Y, 10Y, 30Y

# ATM normal vols (rate units; 0.0070 = 70bp)
atm_normal2 = {
    1/12:{ 1:0.0090,  2:0.0086,  5:0.0079, 10:0.0072, 30:0.0066},
    0.25:{ 1:0.0086,  2:0.0082,  5:0.0074, 10:0.0068, 30:0.0062},
    0.5: { 1:0.0081,  2:0.0077,  5:0.0070, 10:0.0064, 30:0.0059},
    1.0: { 1:0.0074,  2:0.0070,  5:0.0064, 10:0.0059, 30:0.0055},
    2.0: { 1:0.0067,  2:0.0063,  5:0.0058, 10:0.0054, 30:0.0051},
    5.0: { 1:0.0060,  2:0.0057,  5:0.0053, 10:0.0050, 30:0.0048},
    10.0:{ 1:0.0056,  2:0.0053,  5:0.0050, 10:0.0047, 30:0.0045},
}

# SABR-style params that "look like a desk surface":

beta = 0.6    # normal SABR

def nu_surface(expiry, tenor):
    base = {1/12:1.20, 0.25:1.00, 0.5:0.85, 1.0:0.70, 2.0:0.55, 5.0:0.40, 10.0:0.30}[expiry]
    tenor_factor = {1.0:1.00, 2.0:0.95, 5.0:0.90, 10.0:0.85, 30.0:0.80}[tenor]
    return base * tenor_factor

def rho_surface(expiry, tenor):
    base = {1/12:-0.45, 0.25:-0.42, 0.5:-0.38, 1.0:-0.34, 2.0:-0.30, 5.0:-0.26, 10.0:-0.22}[expiry]
    tenor_adj = {1.0:0.00, 2.0:0.02, 5.0:0.04, 10.0:0.05, 30.0:0.06}[tenor]
    return base + tenor_adj

rows = []
for e in expiries:
    for t in tenors:
        rows.append([
            "SOFR-1B",
            float(e),
            float(t),
            float(atm_normal2[e][t]),
            beta,
            float(nu_surface(e, t)),
            float(rho_surface(e, t)),
        ])

sabr_sofr2 = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])
sabr_sofr2


,INDEX,AXIS1,AXIS2,NORMALVOL,BETA,NU,RHO
0,SOFR-1B,0.083333,1.000000,0.009000,0.600000,1.200000,-0.450000
1,SOFR-1B,0.083333,2.000000,0.008600,0.600000,1.140000,-0.430000
2,SOFR-1B,0.083333,5.000000,0.007900,0.600000,1.080000,-0.410000
3,SOFR-1B,0.083333,10.000000,0.007200,0.600000,1.020000,-0.400000
4,SOFR-1B,0.083333,30.000000,0.006600,0.600000,0.960000,-0.390000
5,SOFR-1B,0.250000,1.000000,0.008600,0.600000,1.000000,-0.420000
6,SOFR-1B,0.250000,2.000000,0.008200,0.600000,0.950000,-0.400000
7,SOFR-1B,0.250000,5.000000,0.007400,0.600000,0.900000,-0.380000
8,SOFR-1B,0.250000,10.000000,0.006800,0.600000,0.850000,-0.370000
9,SOFR-1B,0.250000,30.000000,0.006200,0.600000,0.800000,-0.360000


In [24]:
PARAMS = ["NORMALVOL","BETA","NU","RHO"]

sabr_objs = []
for idx_name, sub in sabr_sofr2.groupby("INDEX"):
    for param in PARAMS:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0)
               .sort_index(axis=1)
        )
        sabr_objs.append(
            Data2D.createDataObject(
                data_type=param,          # stored lower() internally
                data_convention=idx_name, # e.g. "SOFR-1B"
                df=pivot
            )
        )

sabr_dc3 = DataCollection(sabr_objs)

In [25]:
value_date = "2025-09-24"

sabr_build_methods = [
    {
        "TARGET": idx_name,         # "SOFR-1B"
        "VALUES": param,            # "NORMALVOL", "BETA", "NU", "RHO"
        "INTERPOLATION": "LINEAR",
        "SHIFT": 0.0,
        "VOL_DECAY_SPEED": 0.2
    }
    for idx_name in sabr_sofr2["INDEX"].unique()
    for param in PARAMS
]

sabr_model3 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=sabr_dc3,
    buildMethodCollection=sabr_build_methods,
    ycModel=yc
)

print(sabr_model3.components.keys())

dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [26]:
swaption = ProductOvernightSwaption(
    optionExpiry="2026-09-24",
    swapStart="2026-09-24",
    swapEnd="2031-09-24",
    frequency="1Y",
    overnightIndex="SOFR-1B",
    optionType="PAYER",
    strikeRate=0.04,
    notional=10000000,
    longOrShort="LONG",
)

vp = {"SABR_METHOD": "hagan", "FUNDING INDEX": "SOFR-1B"}  # method will be forced to Hagan anyway
ve = ValuationEngineRegistry().new_valuation_engine(sabr_model3, vp, swaption)

ve.calculateValue()

c= ve.value[1]


In [27]:
(c-a)/0.0001

np.float64(15071147.171518385)

In [28]:
import pandas as pd

# Expiry x Tenor grid (years)
expiries = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0, 10.0]      # 1M, 3M, 6M, 1Y, 2Y, 5Y, 10Y
tenors   = [1.0, 2.0, 5.0, 10.0, 30.0]                 # 1Y, 2Y, 5Y, 10Y, 30Y

# ATM normal vols (rate units; 0.0070 = 70bp)
atm_normal2 = {
    1/12:{ 1:0.0090,  2:0.0086,  5:0.0079, 10:0.0072, 30:0.0066},
    0.25:{ 1:0.0086,  2:0.0082,  5:0.0074, 10:0.0068, 30:0.0062},
    0.5: { 1:0.0081,  2:0.0077,  5:0.0070, 10:0.0064, 30:0.0059},
    1.0: { 1:0.0074,  2:0.0070,  5:0.0063, 10:0.0059, 30:0.0055},
    2.0: { 1:0.0067,  2:0.0063,  5:0.0058, 10:0.0054, 30:0.0051},
    5.0: { 1:0.0060,  2:0.0057,  5:0.0053, 10:0.0050, 30:0.0048},
    10.0:{ 1:0.0056,  2:0.0053,  5:0.0050, 10:0.0047, 30:0.0045},
}

# SABR-style params that "look like a desk surface":

beta = 0.61

def nu_surface(expiry, tenor):
    base = {1/12:1.20, 0.25:1.00, 0.5:0.85, 1.0:0.70, 2.0:0.55, 5.0:0.40, 10.0:0.30}[expiry]
    tenor_factor = {1.0:1.00, 2.0:0.95, 5.0:0.90, 10.0:0.85, 30.0:0.80}[tenor]
    return base * tenor_factor

def rho_surface(expiry, tenor):
    base = {1/12:-0.45, 0.25:-0.42, 0.5:-0.38, 1.0:-0.34, 2.0:-0.30, 5.0:-0.26, 10.0:-0.22}[expiry]
    tenor_adj = {1.0:0.00, 2.0:0.02, 5.0:0.04, 10.0:0.05, 30.0:0.06}[tenor]
    return base + tenor_adj

rows = []
for e in expiries:
    for t in tenors:
        rows.append([
            "SOFR-1B",
            float(e),
            float(t),
            float(atm_normal2[e][t]),
            beta,
            float(nu_surface(e, t)),
            float(rho_surface(e, t)),
        ])

sabr_sofr3 = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])
sabr_sofr3


,INDEX,AXIS1,AXIS2,NORMALVOL,BETA,NU,RHO
0,SOFR-1B,0.083333,1.000000,0.009000,0.610000,1.200000,-0.450000
1,SOFR-1B,0.083333,2.000000,0.008600,0.610000,1.140000,-0.430000
2,SOFR-1B,0.083333,5.000000,0.007900,0.610000,1.080000,-0.410000
3,SOFR-1B,0.083333,10.000000,0.007200,0.610000,1.020000,-0.400000
4,SOFR-1B,0.083333,30.000000,0.006600,0.610000,0.960000,-0.390000
5,SOFR-1B,0.250000,1.000000,0.008600,0.610000,1.000000,-0.420000
6,SOFR-1B,0.250000,2.000000,0.008200,0.610000,0.950000,-0.400000
7,SOFR-1B,0.250000,5.000000,0.007400,0.610000,0.900000,-0.380000
8,SOFR-1B,0.250000,10.000000,0.006800,0.610000,0.850000,-0.370000
9,SOFR-1B,0.250000,30.000000,0.006200,0.610000,0.800000,-0.360000


In [29]:
PARAMS = ["NORMALVOL","BETA","NU","RHO"]

sabr_objs = []
for idx_name, sub in sabr_sofr3.groupby("INDEX"):
    for param in PARAMS:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0)
               .sort_index(axis=1)
        )
        sabr_objs.append(
            Data2D.createDataObject(
                data_type=param,          # stored lower() internally
                data_convention=idx_name, # e.g. "SOFR-1B"
                df=pivot
            )
        )

sabr_dc4 = DataCollection(sabr_objs)

In [30]:
value_date = "2025-09-24"

sabr_build_methods = [
    {
        "TARGET": idx_name,         # "SOFR-1B"
        "VALUES": param,            # "NORMALVOL", "BETA", "NU", "RHO"
        "INTERPOLATION": "LINEAR",
        "SHIFT": 0.0,
        "VOL_DECAY_SPEED": 0.2
    }
    for idx_name in sabr_sofr3["INDEX"].unique()
    for param in PARAMS
]

sabr_model4 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=sabr_dc4,
    buildMethodCollection=sabr_build_methods,
    ycModel=yc
)

print(sabr_model4.components.keys())

dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [31]:
swaption = ProductOvernightSwaption(
    optionExpiry="2026-09-24",
    swapStart="2026-09-24",
    swapEnd="2031-09-24",
    frequency="1Y",
    overnightIndex="SOFR-1B",
    optionType="PAYER",
    strikeRate=0.04,
    notional=10000000,
    longOrShort="LONG",
)

vp = {"SABR_METHOD": "hagan", "FUNDING INDEX": "SOFR-1B"}  # method will be forced to Hagan anyway
ve = ValuationEngineRegistry().new_valuation_engine(sabr_model4, vp, swaption)

ve.calculateValue()

d= ve.value[1]


In [32]:
(d-a)/0.01

np.float64(3942.0129906997317)

In [33]:
import pandas as pd

# Expiry x Tenor grid (years)
expiries = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0, 10.0]      # 1M, 3M, 6M, 1Y, 2Y, 5Y, 10Y
tenors   = [1.0, 2.0, 5.0, 10.0, 30.0]                 # 1Y, 2Y, 5Y, 10Y, 30Y

# ATM normal vols (rate units; 0.0070 = 70bp)
atm_normal2 = {
    1/12:{ 1:0.0090,  2:0.0086,  5:0.0079, 10:0.0072, 30:0.0066},
    0.25:{ 1:0.0086,  2:0.0082,  5:0.0074, 10:0.0068, 30:0.0062},
    0.5: { 1:0.0081,  2:0.0077,  5:0.0070, 10:0.0064, 30:0.0059},
    1.0: { 1:0.0074,  2:0.0070,  5:0.0063, 10:0.0059, 30:0.0055},
    2.0: { 1:0.0067,  2:0.0063,  5:0.0058, 10:0.0054, 30:0.0051},
    5.0: { 1:0.0060,  2:0.0057,  5:0.0053, 10:0.0050, 30:0.0048},
    10.0:{ 1:0.0056,  2:0.0053,  5:0.0050, 10:0.0047, 30:0.0045},
}

# SABR-style params that "look like a desk surface":

beta = 0.6   # normal SABR

def nu_surface(expiry, tenor):
    base = {1/12:1.20, 0.25:1.00, 0.5:0.85, 1.0:0.70, 2.0:0.55, 5.0:0.40, 10.0:0.30}[expiry]
    tenor_factor = {1.0:1.00, 2.0:0.95, 5.0:0.90, 10.0:0.85, 30.0:0.80}[tenor]
    return base * tenor_factor

def rho_surface(expiry, tenor):
    base = {1/12:-0.45, 0.25:-0.42, 0.5:-0.38, 1.0:-0.34, 2.0:-0.30, 5.0:-0.26, 10.0:-0.22}[expiry]
    tenor_adj = {1.0:0.00, 2.0:0.02, 5.0:0.04, 10.0:0.05, 30.0:0.06}[tenor]
    return base + tenor_adj

rows = []
for e in expiries:
    for t in tenors:
        rows.append([
            "SOFR-1B",
            float(e),
            float(t),
            float(atm_normal2[e][t]),
            beta,
            float(nu_surface(e, t)),
            float(rho_surface(e, t)),
        ])

sabr_sofr4 = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

m = (
    (sabr_sofr4["INDEX"] == "SOFR-1B") &
    np.isclose(sabr_sofr4["AXIS1"], 1.0) &
    np.isclose(sabr_sofr4["AXIS2"], 5.0)
)

sabr_sofr4.loc[m, "RHO"] += 0.01

sabr_sofr4


,INDEX,AXIS1,AXIS2,NORMALVOL,BETA,NU,RHO
0,SOFR-1B,0.083333,1.000000,0.009000,0.600000,1.200000,-0.450000
1,SOFR-1B,0.083333,2.000000,0.008600,0.600000,1.140000,-0.430000
2,SOFR-1B,0.083333,5.000000,0.007900,0.600000,1.080000,-0.410000
3,SOFR-1B,0.083333,10.000000,0.007200,0.600000,1.020000,-0.400000
4,SOFR-1B,0.083333,30.000000,0.006600,0.600000,0.960000,-0.390000
5,SOFR-1B,0.250000,1.000000,0.008600,0.600000,1.000000,-0.420000
6,SOFR-1B,0.250000,2.000000,0.008200,0.600000,0.950000,-0.400000
7,SOFR-1B,0.250000,5.000000,0.007400,0.600000,0.900000,-0.380000
8,SOFR-1B,0.250000,10.000000,0.006800,0.600000,0.850000,-0.370000
9,SOFR-1B,0.250000,30.000000,0.006200,0.600000,0.800000,-0.360000


In [34]:
PARAMS = ["NORMALVOL","BETA","NU","RHO"]

sabr_objs = []
for idx_name, sub in sabr_sofr4.groupby("INDEX"):
    for param in PARAMS:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0)
               .sort_index(axis=1)
        )
        sabr_objs.append(
            Data2D.createDataObject(
                data_type=param,          # stored lower() internally
                data_convention=idx_name, # e.g. "SOFR-1B"
                df=pivot
            )
        )

sabr_dc5 = DataCollection(sabr_objs)

In [35]:
value_date = "2025-09-24"

sabr_build_methods = [
    {
        "TARGET": idx_name,         # "SOFR-1B"
        "VALUES": param,            # "NORMALVOL", "BETA", "NU", "RHO"
        "INTERPOLATION": "LINEAR",
        "SHIFT": 0.0,
        "VOL_DECAY_SPEED": 0.2
    }
    for idx_name in sabr_sofr4["INDEX"].unique()
    for param in PARAMS
]

sabr_model5 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=sabr_dc5,
    buildMethodCollection=sabr_build_methods,
    ycModel=yc
)

print(sabr_model5.components.keys())

dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [36]:
swaption = ProductOvernightSwaption(
    optionExpiry="2026-09-24",
    swapStart="2026-09-24",
    swapEnd="2031-09-24",
    frequency="1Y",
    overnightIndex="SOFR-1B",
    optionType="PAYER",
    strikeRate=0.04,
    notional=10000000,
    longOrShort="LONG",
)

vp = {"SABR_METHOD": "hagan", "FUNDING INDEX": "SOFR-1B"}  # method will be forced to Hagan anyway
ve = ValuationEngineRegistry().new_valuation_engine(sabr_model5, vp, swaption)

ve.calculateValue()

e= ve.value[1]


In [37]:
(e-a)/0.01

np.float64(15528.253824912099)

In [38]:
import pandas as pd

# Expiry x Tenor grid (years)
expiries = [1/12, 0.25, 0.5, 1.0, 2.0, 5.0, 10.0]      # 1M, 3M, 6M, 1Y, 2Y, 5Y, 10Y
tenors   = [1.0, 2.0, 5.0, 10.0, 30.0]                 # 1Y, 2Y, 5Y, 10Y, 30Y

# ATM normal vols (rate units; 0.0070 = 70bp)
atm_normal2 = {
    1/12:{ 1:0.0090,  2:0.0086,  5:0.0079, 10:0.0072, 30:0.0066},
    0.25:{ 1:0.0086,  2:0.0082,  5:0.0074, 10:0.0068, 30:0.0062},
    0.5: { 1:0.0081,  2:0.0077,  5:0.0070, 10:0.0064, 30:0.0059},
    1.0: { 1:0.0074,  2:0.0070,  5:0.0063, 10:0.0059, 30:0.0055},
    2.0: { 1:0.0067,  2:0.0063,  5:0.0058, 10:0.0054, 30:0.0051},
    5.0: { 1:0.0060,  2:0.0057,  5:0.0053, 10:0.0050, 30:0.0048},
    10.0:{ 1:0.0056,  2:0.0053,  5:0.0050, 10:0.0047, 30:0.0045},
}

# SABR-style params that "look like a desk surface":

beta = 0.6   # normal SABR

def nu_surface(expiry, tenor):
    base = {1/12:1.20, 0.25:1.00, 0.5:0.85, 1.0:0.70, 2.0:0.55, 5.0:0.40, 10.0:0.30}[expiry]
    tenor_factor = {1.0:1.00, 2.0:0.95, 5.0:0.90, 10.0:0.85, 30.0:0.80}[tenor]
    return base * tenor_factor

def rho_surface(expiry, tenor):
    base = {1/12:-0.45, 0.25:-0.42, 0.5:-0.38, 1.0:-0.34, 2.0:-0.30, 5.0:-0.26, 10.0:-0.22}[expiry]
    tenor_adj = {1.0:0.00, 2.0:0.02, 5.0:0.04, 10.0:0.05, 30.0:0.06}[tenor]
    return base + tenor_adj

rows = []
for e in expiries:
    for t in tenors:
        rows.append([
            "SOFR-1B",
            float(e),
            float(t),
            float(atm_normal2[e][t]),
            beta,
            float(nu_surface(e, t)),
            float(rho_surface(e, t)),
        ])

sabr_sofr5 = pd.DataFrame(rows, columns=["INDEX","AXIS1","AXIS2","NORMALVOL","BETA","NU","RHO"])

m = (
    (sabr_sofr5["INDEX"] == "SOFR-1B") &
    np.isclose(sabr_sofr5["AXIS1"], 1.0) &
    np.isclose(sabr_sofr5["AXIS2"], 5.0)
)

sabr_sofr5.loc[m, "NU"] += 0.01

sabr_sofr5


,INDEX,AXIS1,AXIS2,NORMALVOL,BETA,NU,RHO
0,SOFR-1B,0.083333,1.000000,0.009000,0.600000,1.200000,-0.450000
1,SOFR-1B,0.083333,2.000000,0.008600,0.600000,1.140000,-0.430000
2,SOFR-1B,0.083333,5.000000,0.007900,0.600000,1.080000,-0.410000
3,SOFR-1B,0.083333,10.000000,0.007200,0.600000,1.020000,-0.400000
4,SOFR-1B,0.083333,30.000000,0.006600,0.600000,0.960000,-0.390000
5,SOFR-1B,0.250000,1.000000,0.008600,0.600000,1.000000,-0.420000
6,SOFR-1B,0.250000,2.000000,0.008200,0.600000,0.950000,-0.400000
7,SOFR-1B,0.250000,5.000000,0.007400,0.600000,0.900000,-0.380000
8,SOFR-1B,0.250000,10.000000,0.006800,0.600000,0.850000,-0.370000
9,SOFR-1B,0.250000,30.000000,0.006200,0.600000,0.800000,-0.360000


In [39]:
PARAMS = ["NORMALVOL","BETA","NU","RHO"]

sabr_objs = []
for idx_name, sub in sabr_sofr5.groupby("INDEX"):
    for param in PARAMS:
        pivot = (
            sub.pivot(index="AXIS1", columns="AXIS2", values=param)
               .sort_index(axis=0)
               .sort_index(axis=1)
        )
        sabr_objs.append(
            Data2D.createDataObject(
                data_type=param,          # stored lower() internally
                data_convention=idx_name, # e.g. "SOFR-1B"
                df=pivot
            )
        )

sabr_dc6 = DataCollection(sabr_objs)

In [40]:
value_date = "2025-09-24"

sabr_build_methods = [
    {
        "TARGET": idx_name,         # "SOFR-1B"
        "VALUES": param,            # "NORMALVOL", "BETA", "NU", "RHO"
        "INTERPOLATION": "LINEAR",
        "SHIFT": 0.0,
        "VOL_DECAY_SPEED": 0.2
    }
    for idx_name in sabr_sofr5["INDEX"].unique()
    for param in PARAMS
]

sabr_model6 = SabrModel.from_curve(
    valueDate=value_date,
    dataCollection=sabr_dc6,
    buildMethodCollection=sabr_build_methods,
    ycModel=yc
)

print(sabr_model6.components.keys())

dict_keys(['SOFR-1B-NORMALVOL', 'SOFR-1B-BETA', 'SOFR-1B-NU', 'SOFR-1B-RHO'])


In [41]:
swaption = ProductOvernightSwaption(
    optionExpiry="2026-09-24",
    swapStart="2026-09-24",
    swapEnd="2031-09-24",
    frequency="1Y",
    overnightIndex="SOFR-1B",
    optionType="PAYER",
    strikeRate=0.04,
    notional=10000000,
    longOrShort="LONG",
)

vp = {"SABR_METHOD": "hagan", "FUNDING INDEX": "SOFR-1B"}  # method will be forced to Hagan anyway
ve = ValuationEngineRegistry().new_valuation_engine(sabr_model6, vp, swaption)

ve.calculateValue()

f= ve.value[1]


In [42]:
(a-f)/0.01

np.float64(2350.3036777241505)